In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
bbbp_chemberta2_valid2 = pd.read_csv('./chemberta2/results/bbbp/chemberta2_valid2_bbbp_3_predictions.csv')
bbbp_molformer_valid2 = pd.read_csv('./molformer/results/bbbp/molformer_valid2_bbbp_3_epoch29.csv')
bbbp_molbert_valid2 = pd.read_csv('./molbert/results/bbbp/molbert_valid2_bbbp_3.csv')

# Load the test data for each model
bbbp_chemberta2_test = pd.read_csv('./chemberta2/results/bbbp/chemberta2_test_bbbp_3_predictions.csv')
bbbp_molformer_test = pd.read_csv('./molformer/results/bbbp/molformer_test_bbbp_3_epoch29.csv')
bbbp_molbert_test = pd.read_csv('./molbert/results/bbbp/molbert_test_bbbp_3.csv')

# features

# Load the features from chemberta
bbbp_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/bbbp/chemberta2_valid2_bbbp_3_features.csv')
bbbp_chemberta2_features_test = pd.read_csv('./chemberta2/features/bbbp/chemberta2_test_bbbp_3_features.csv')

# Load the features from molformer
bbbp_molformer_features_valid2 = pd.read_csv('./molformer/features/bbbp/molformer_valid2_bbbp_3_features.csv')
bbbp_molformer_features_test = pd.read_csv('./molformer/features/bbbp/molformer_test_bbbp_3_features.csv')

# Load the features from molbert
bbbp_molbert_features_valid2 = pd.read_csv('./molbert/features/bbbp/molbert_valid2_bbbp_3_features.csv')
bbbp_molbert_features_test = pd.read_csv('./molbert/features/bbbp/molbert_test_bbbp_3_features.csv')

In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
bbbp_chemberta_actual = bbbp_chemberta2_test['p_np']
bbbp_chemberta_pred = bbbp_chemberta2_test['y_pred']
bbbp_chemberta_probs = bbbp_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
bbbp_molformer_actual = bbbp_molformer_test['Actual']
bbbp_molformer_pred = (bbbp_molformer_test['Prob_Class_1'] > 0.5).astype(int)
bbbp_molformer_probs = bbbp_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
bbbp_molbert_actual = bbbp_molbert_test['target']
bbbp_molbert_pred = bbbp_molbert_test['pred']
bbbp_molbert_probs = bbbp_molbert_test['prob']

# Calculating metrics
bbbp_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", bbbp_chemberta_actual, bbbp_chemberta_pred, bbbp_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", bbbp_molformer_actual, bbbp_molformer_pred, bbbp_molformer_probs['Prob_Class_1']),
                                         ("Molbert", bbbp_molbert_actual, bbbp_molbert_pred, bbbp_molbert_probs)]:
    bbbp_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

bbbp_metrics_results

{'Chemberta2': {'Accuracy': 0.8292682926829268,
  'F1 Score': 0.8844884488448845,
  'ROC-AUC': 0.9488724127278345,
  'PR-AUC': 0.9882303014277589},
 'Molformer': {'Accuracy': 0.8774509803921569,
  'F1 Score': 0.924924924924925,
  'ROC-AUC': 0.8789632213062777,
  'PR-AUC': 0.965741487133955},
 'Molbert': {'Accuracy': 0.9068627450980392,
  'F1 Score': 0.9425981873111783,
  'ROC-AUC': 0.9514901712111604,
  'PR-AUC': 0.9885586344415213}}

In [3]:
# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molformer_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molformer_valid2['smiles'])
print(f"Missing smiles in molformer_valid2: {missing_smiles_molformer_valid2}")

# Identify the 'smiles' values in chemberta2_valid2 that are not in molbert_valid2
missing_smiles_molbert_valid2 = set(bbbp_chemberta2_valid2['smiles']) - set(bbbp_molbert_valid2['smiles'])
print(f"Missing smiles in molbert_valid2: {missing_smiles_molbert_valid2}")

# Combine the invalid indices from molformer_valid2 with the missing indices from molbert_valid2
combined_invalid_smiles_valid2 = list(set(missing_smiles_molformer_valid2).union(set(missing_smiles_molbert_valid2)))

print(f"These indices will be removed from the valid2 set: {combined_invalid_smiles_valid2}")

Missing smiles in molformer_valid2: {'n1c(csc1\\[NH]=C(\\N)N)c1ccccc1', 'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br', 's1cc(nc1\\[NH]=C(\\N)N)C', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', 'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC'}
Missing smiles in molbert_valid2: {'c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC', 'n1c(csc1\\[NH]=C(\\N)N)c1ccccc1', '[Na+].Cc1sc(SCC2=C(N3[C@H](SC2)[C@H](NC(=O)Cn4cnnn4)C3=O)C([O-])=O)nn1', '[Na+].CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](NC(=O)C3=CC=C(NC3=O)c4ccc(cc4)[S](=O)(=O)N(CCO)CCO)c5ccc(O)cc5)C(=O)N2[C@H]1C([O-])=O', 'c1c(c(ncc1)CSCCN\\C(=[NH]\\C#N)NCC)Br', 's1cc(CSCCN\\C(NC)=[NH]\\C#N)nc1\\[NH]=C(\\N)N', '[Na+].CO\\N=C(C(=O)N[C@@H]1[C@@H]2SCC(=C(N2C1=O)C([O-])=O)COC(C)=O)\\c3csc(N)n3', '[Na+].Cn1nnnc1SCC2=C(N3[C@H](SC2)[C@H](NC(=O)CSC(F)(F)F)C3=O)C([O-])=O', 's1cc(nc1\\[NH]=C(\\N)N)C'}
These indices will be removed from the valid2 set: ['n1c(csc1\\[NH]=C(\\N)N)c1ccccc1', '[Na+].Cc1sc(SCC2=C(N3[C@H](SC2)[C@H](NC(=O)Cn4cnnn4)C3=O)C([O-])=O)nn1', '[Na+].CC1(C)S[

In [4]:
# Function to remove invalid SMILES
def remove_invalid_smiles(df, invalid_smiles_list):
    return df[~df['smiles'].isin(invalid_smiles_list)]

# Remove invalid SMILES from each dataframe
bbbp_chemberta2_valid2 = remove_invalid_smiles(bbbp_chemberta2_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_valid2 = remove_invalid_smiles(bbbp_molformer_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_valid2 = remove_invalid_smiles(bbbp_molbert_valid2, combined_invalid_smiles_valid2)
bbbp_chemberta2_features_valid2 = remove_invalid_smiles(bbbp_chemberta2_features_valid2, combined_invalid_smiles_valid2)
bbbp_molformer_features_valid2 = remove_invalid_smiles(bbbp_molformer_features_valid2, combined_invalid_smiles_valid2)
bbbp_molbert_features_valid2 = remove_invalid_smiles(bbbp_molbert_features_valid2, combined_invalid_smiles_valid2)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_valid2.shape)
print(bbbp_molformer_valid2.shape)
print(bbbp_molbert_valid2.shape)
print(bbbp_chemberta2_features_valid2.shape)
print(bbbp_molformer_features_valid2.shape)
print(bbbp_molbert_features_valid2.shape)

(401, 8)
(401, 6)
(401, 4)
(401, 386)
(401, 769)
(401, 769)


In [5]:
bbbp_y_ensemble_valid2 = bbbp_chemberta2_valid2['p_np']

# Convert the ensemble target to a Series if not already done
bbbp_y_ensemble_valid2_s = pd.Series(bbbp_y_ensemble_valid2).reset_index(drop=True)

# Create dataframes for each model's class 1 probability
bbbp_chemberta2_prob = pd.DataFrame({'chemberta2': bbbp_chemberta2_valid2['softmax_class_1_prob']})
bbbp_chemberta2_prob.reset_index(drop=True, inplace=True)

bbbp_molformer_prob = pd.DataFrame({'molformer': bbbp_molformer_valid2['Prob_Class_1']})
bbbp_molformer_prob.reset_index(drop=True, inplace=True)

bbbp_molbert_prob = pd.DataFrame({'molbert': bbbp_molbert_valid2['prob']})
bbbp_molbert_prob.reset_index(drop=True, inplace=True)

# do the same for features bbbp_chemberta2_features_valid2.iloc[:, 2:]
bbbp_chemberta2_features = pd.DataFrame(bbbp_chemberta2_features_valid2.iloc[:, 2:])
bbbp_chemberta2_features.reset_index(drop=True, inplace=True)

bbbp_molformer_features = pd.DataFrame(bbbp_molformer_features_valid2.iloc[:, 1:])
bbbp_molformer_features.reset_index(drop=True, inplace=True)

bbbp_molbert_features = pd.DataFrame(bbbp_molbert_features_valid2.iloc[:, 1:])
bbbp_molbert_features.reset_index(drop=True, inplace=True)

# bbbp_features = pd.concat([bbbp_chemberta2_features, bbbp_molformer_features, bbbp_molbert_features], axis=1)

# Combine probabilities into one dataframe
train_bbbp_prob = pd.concat([bbbp_chemberta2_prob, bbbp_molformer_prob, bbbp_molbert_prob], axis=1)

# Function to calculate BCE for each row with epsilon to avoid log(0)
def calculate_bce_rowwise(y_true, y_pred, epsilon=1e-12):
    # Clip y_pred to ensure it's between epsilon and 1-epsilon
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Calculate row-wise BCE for each model
bce_chemberta = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_chemberta2_prob['chemberta2'])
bce_molformer = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molformer_prob['molformer'])
bce_molbert = calculate_bce_rowwise(bbbp_y_ensemble_valid2_s, bbbp_molbert_prob['molbert'])

# Create a dataframe for row-wise BCE losses
bce_loss_df = pd.DataFrame({
    'bce_chemberta': bce_chemberta,
    'bce_molformer': bce_molformer,
    'bce_molbert': bce_molbert
})

# Final ensemble X matrix: Combine row-wise BCE losses, predictions, and features
bbbp_X_ensemble_valid2 = pd.concat([bce_loss_df, train_bbbp_prob], axis=1)

In [6]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss
import numpy as np

# Set seeds for reproducibility
np.random.seed(0)

# Combine probabilities with their respective feature sets
chemberta_X = pd.concat([bbbp_chemberta2_prob, bbbp_chemberta2_features], axis=1)
molformer_X = pd.concat([bbbp_molformer_prob, bbbp_molformer_features], axis=1)
molbert_X = pd.concat([bbbp_molbert_prob, bbbp_molbert_features], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Define the binary cross-entropy loss values as target variables (y)
chemberta_y_bce = bce_chemberta  # Row-wise BCE loss calculated earlier
molformer_y_bce = bce_molformer  # Row-wise BCE loss calculated earlier
molbert_y_bce = bce_molbert      # Row-wise BCE loss calculated earlier

# Define RMSE scorer
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Define the hyperparameter space for regression using continuous distributions
xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Objective function for hyperopt with XGBRegressor
def objective(params, X_train, y_train):
    # Convert float outputs of hp.quniform to int for certain parameters
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    # Initialize XGBRegressor with the given parameters
    model = xgb.XGBRegressor(**params, random_state=0)
    
    # RMSE as the scoring metric
    rmse = make_scorer(rmse_scorer, greater_is_better=False)
    score = cross_val_score(model, X_train, y_train, scoring=rmse, cv=5)
    
    # Minimize the negative RMSE score
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Function to optimize and train the model for each dataset
def optimize_and_train(X_train, y_train):
    # Run the Bayesian optimization
    trials = Trials()
    best_params = fmin(fn=lambda params: objective(params, X_train, y_train), 
                       space=xgb_hyperopt_space, 
                       algo=tpe.suggest, 
                       max_evals=50, 
                       trials=trials,
                       rstate=np.random.default_rng(0),  # Seed for reproducibility in hyperopt
                       early_stop_fn=no_progress_loss(10))

    # Convert the best hyperparameters to the correct data types
    best_params['n_estimators'] = int(best_params['n_estimators'])
    best_params['max_depth'] = int(best_params['max_depth'])

    # Initialize and train the XGBoost model with the best parameters
    model = xgb.XGBRegressor(**best_params, random_state=0)
    model.fit(X_train, y_train)
    
    return model

# Optimizing and training for chemberta dataset
print("Optimizing for chemberta dataset...")
best_model_chemberta = optimize_and_train(chemberta_X_scaled, chemberta_y_bce)

# Optimizing and training for molformer dataset
print("Optimizing for molformer dataset...")
best_model_molformer = optimize_and_train(molformer_X_scaled, molformer_y_bce)

# Optimizing and training for molbert dataset
print("Optimizing for molbert dataset...")
best_model_molbert = optimize_and_train(molbert_X_scaled, molbert_y_bce)

# Print a confirmation message
print("Models trained for chemberta, molformer, and molbert datasets.")


Optimizing for chemberta dataset...
 44%|████▍     | 22/50 [02:14<02:51,  6.12s/trial, best loss: 0.28087251382773964]
Optimizing for molformer dataset...
 46%|████▌     | 23/50 [04:47<05:37, 12.49s/trial, best loss: 3.5584923373347563]
Optimizing for molbert dataset...
 32%|███▏      | 16/50 [02:51<06:04, 10.72s/trial, best loss: 1.5532111240224733]
Models trained for chemberta, molformer, and molbert datasets.


In [7]:
# identify missing in test
missing_smiles_molformer_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molformer_test['smiles'])
print(f"Missing smiles in molformer_test: {missing_smiles_molformer_test}")

missing_smiles_molbert_test = set(bbbp_chemberta2_test['smiles']) - set(bbbp_molbert_test['smiles'])
print(f"Missing smiles in molbert_test: {missing_smiles_molbert_test}")

# Combine the invalid smiles from molformer_test with the missing smiles from molbert_test
combined_invalid_smiles_test = list(set(missing_smiles_molformer_test).union(set(missing_smiles_molbert_test)))
print(f"These indices will be removed from the test set: {combined_invalid_smiles_test}")

Missing smiles in molformer_test: {'n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O'}
Missing smiles in molbert_test: {'n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O'}
These indices will be removed from the test set: ['n1c(csc1\\[NH]=C(\\N)N)c1cccc(c1)NC(C)=O']


In [8]:
# Remove invalid SMILES from each dataframe
bbbp_chemberta2_test = remove_invalid_smiles(bbbp_chemberta2_test, combined_invalid_smiles_test)
bbbp_molformer_test = remove_invalid_smiles(bbbp_molformer_test, combined_invalid_smiles_test)
bbbp_molbert_test = remove_invalid_smiles(bbbp_molbert_test, combined_invalid_smiles_test)
bbbp_chemberta2_features_test = remove_invalid_smiles(bbbp_chemberta2_features_test, combined_invalid_smiles_test)
bbbp_molformer_features_test = remove_invalid_smiles(bbbp_molformer_features_test, combined_invalid_smiles_test)
bbbp_molbert_features_test = remove_invalid_smiles(bbbp_molbert_features_test, combined_invalid_smiles_test)

# Print the shapes of the dataframes after removal
print(bbbp_chemberta2_test.shape)
print(bbbp_molformer_test.shape)
print(bbbp_molbert_test.shape)
print(bbbp_chemberta2_features_test.shape)
print(bbbp_molformer_features_test.shape)
print(bbbp_molbert_features_test.shape)

(204, 8)
(204, 6)
(204, 4)
(204, 386)
(204, 769)
(204, 769)


In [9]:
import numpy as np
from sklearn.metrics import log_loss

# Test data for each model
bbbp_chemberta2_prob_test = pd.DataFrame({'chemberta2': bbbp_chemberta2_test['softmax_class_1_prob']})
bbbp_chemberta2_prob_test.reset_index(drop=True, inplace=True)

bbbp_molformer_prob_test = pd.DataFrame({'molformer': bbbp_molformer_test['Prob_Class_1']})
bbbp_molformer_prob_test.reset_index(drop=True, inplace=True)

bbbp_molbert_prob_test = pd.DataFrame({'molbert': bbbp_molbert_test['prob']})
bbbp_molbert_prob_test.reset_index(drop=True, inplace=True)

bbbp_chemberta2_features_t = pd.DataFrame(bbbp_chemberta2_features_test.iloc[:, 2:])
bbbp_chemberta2_features_t.reset_index(drop=True, inplace=True)

bbbp_molformer_features_t  = pd.DataFrame(bbbp_molformer_features_test.iloc[:, 1:])
bbbp_molformer_features_t.reset_index(drop=True, inplace=True)

bbbp_molbert_features_t = pd.DataFrame(bbbp_molbert_features_test.iloc[:, 1:])
bbbp_molbert_features_t.reset_index(drop=True, inplace=True)

# Combine probabilities with the respective feature sets for the test set
chemberta_X_test = pd.concat([bbbp_chemberta2_prob_test, bbbp_chemberta2_features_t], axis=1)
molformer_X_test = pd.concat([bbbp_molformer_prob_test, bbbp_molformer_features_t], axis=1)
molbert_X_test = pd.concat([bbbp_molbert_prob_test, bbbp_molbert_features_t], axis=1)

# Standardize the test set based on the previously fitted scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Predict using the best models from valid2
chemberta_pred_test = best_model_chemberta.predict(chemberta_X_test_scaled)
molformer_pred_test = best_model_molformer.predict(molformer_X_test_scaled)
molbert_pred_test = best_model_molbert.predict(molbert_X_test_scaled)

# Convert the predictions (numpy arrays) to pandas Series
chemberta_pred_test_series = pd.Series(chemberta_pred_test, name='bce_chemberta')
molformer_pred_test_series = pd.Series(molformer_pred_test, name='bce_molformer')
molbert_pred_test_series = pd.Series(molbert_pred_test, name='bce_molbert')

# Now concatenate the series with the test set probabilities
bbbp_X_ensemble_test = pd.concat([
    chemberta_pred_test_series,                     # BCE for Chemberta
    molformer_pred_test_series,                     # BCE for Molformer
    molbert_pred_test_series,                       # BCE for Molbert
    bbbp_chemberta2_prob_test['chemberta2'],        # Chemberta test probabilities
    bbbp_molformer_prob_test['molformer'],          # Molformer test probabilities
    bbbp_molbert_prob_test['molbert']               # Molbert test probabilities
], axis=1)

bbbp_X_ensemble_test.columns = ['bce_chemberta', 'bce_molformer', 'bce_molbert', 'chemberta2', 'molformer', 'molbert']

# optional for evaluation
bbbp_y_ensemble_test = bbbp_chemberta2_test['p_np']

In [10]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
bbbp_X_ensemble_valid2_scaled = scaler.fit_transform(bbbp_X_ensemble_valid2)
bbbp_X_ensemble_test_scaled = scaler.transform(bbbp_X_ensemble_test)

# transform back to dataframe
bbbp_X_ensemble_valid2_scaled = pd.DataFrame(bbbp_X_ensemble_valid2_scaled, columns=bbbp_X_ensemble_valid2.columns)
bbbp_X_ensemble_test_scaled = pd.DataFrame(bbbp_X_ensemble_test_scaled, columns=bbbp_X_ensemble_test.columns)


In [11]:
bbbp_X_ensemble_valid2_selected = bbbp_X_ensemble_valid2_scaled
bbbp_X_ensemble_test_selected = bbbp_X_ensemble_test_scaled

# check shapes
print(bbbp_X_ensemble_valid2_selected.shape)
print(bbbp_X_ensemble_test_selected.shape)

(401, 6)
(204, 6)


In [13]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Set seeds for reproducibility
np.random.seed(0)

# Define the hyperparameter space using continuous distributions
bbbp_xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Convert float outputs of hp.quniform to int for certain parameters
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    model = xgb.XGBClassifier(**params, random_state=0)
    
    # Cross-validated AUC score as the objective
    roc_auc = make_scorer(roc_auc_score, response_method=None)
    score = cross_val_score(model, bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2, scoring=roc_auc, cv=5)
    
    # Minimize the negative ROC AUC score
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Run the Bayesian optimization
trials = Trials()
bbbp_xgb_best_params = fmin(fn=objective, 
                          space=bbbp_xgb_hyperopt_space, 
                          algo=tpe.suggest, 
                          max_evals=50, 
                          trials=trials,
                          rstate=np.random.default_rng(0),  # Seed for hyperopt
                          early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", bbbp_xgb_best_params)

 20%|██        | 10/50 [00:01<00:07,  5.29trial/s, best loss: -0.9839181286549709]
Best hyperparameters: {'colsample_bytree': 0.9526919134012697, 'learning_rate': 0.25068293229696836, 'max_depth': 4.0, 'n_estimators': 100.0, 'subsample': 0.8411318103731124}


In [14]:
# Convert parameters obtained from Hyperopt to the correct data type
bbbp_xgb_best_params['n_estimators'] = int(bbbp_xgb_best_params['n_estimators'])
bbbp_xgb_best_params['max_depth'] = int(bbbp_xgb_best_params['max_depth'])

# Initialize and train the XGBoost model with the best parameters
bbbp_xgb_model = xgb.XGBClassifier(**bbbp_xgb_best_params, random_state=0)
bbbp_xgb_model.fit(bbbp_X_ensemble_valid2_selected, bbbp_y_ensemble_valid2)

# Predict the test set
bbbp_xgb_best_pred = bbbp_xgb_model.predict(bbbp_X_ensemble_test_selected)
bbbp_xgb_best_probs = bbbp_xgb_model.predict_proba(bbbp_X_ensemble_test_selected)

# Calculate the metrics
bbbp_xgb_best_metrics = {
    "Accuracy": accuracy_score(bbbp_y_ensemble_test, bbbp_xgb_best_pred),
    "F1 Score": f1_score(bbbp_y_ensemble_test, bbbp_xgb_best_pred),
    "ROC-AUC": roc_auc_score(bbbp_y_ensemble_test, bbbp_xgb_best_probs[:, 1]),
    "PR-AUC": average_precision_score(bbbp_y_ensemble_test, bbbp_xgb_best_probs[:, 1])
}

bbbp_xgb_best_metrics

{'Accuracy': 0.8970588235294118,
 'F1 Score': 0.9357798165137615,
 'ROC-AUC': 0.9499048826886494,
 'PR-AUC': 0.9871922319956341}

In [15]:
# report all the metrics for bbbp
bbbp_metrics_results["XGBoost"] = bbbp_xgb_best_metrics

bbbp_metrics_df = pd.DataFrame(bbbp_metrics_results).T

# keep 3 digits after the decimal point
bbbp_metrics_df = bbbp_metrics_df.round(3)

# export as csv
bbbp_metrics_df.to_csv('./split3_bbbp_metrics_xgb.csv')